In [1]:
import os
import joblib
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from skimage.color import rgb2gray
from skimage.io import imread


# Define the path to the folder containing the images
folder_path = '/Users/metoditarnev/Desktop/archive(1)/dataset/Train'
validation_path = '/Users/metoditarnev/Desktop/archive(1)/dataset/Validation'

# Initialize empty lists to hold the image data and labels
X = []
y = []
X_val = []
y_val = []

# Iterate over the subdirectories in the folder, where each subdirectory is a label
for label in os.listdir(folder_path):
    label_path = os.path.join(folder_path, label)
    if not os.path.isdir(label_path):
        continue
    # Iterate over the image files in the subdirectory and load each image
    for file_name in os.listdir(label_path):
        if not file_name.endswith('.png'):
            continue
        image_path = os.path.join(label_path, file_name)
        image = imread(image_path)
        features = hog(image,
                       orientations=8,
                       pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2),
                       block_norm='L2-Hys')
        X.append(features)
        y.append(label)

# Convert the X and y lists to numpy arrays
X = np.array(X)
y = np.array(y)

for label in os.listdir(validation_path):
    label_path = os.path.join(validation_path, label)
    if not os.path.isdir(label_path):
        continue
    # Iterate over the image files in the subdirectory and load each image
    for file_name in os.listdir(label_path):
        if not file_name.endswith('.png'):
            continue
        image_path = os.path.join(label_path, file_name)
        image = imread(image_path)
        features = hog(image,
                       orientations=8,
                       pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2),
                       block_norm='L2-Hys')
        X_val.append(features)
        y_val.append(label)

# Convert the X_val and y_val lists to numpy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC())
])

# Define the parameter grid to search over
param_grid = {'svc__C': [0.001, 0.01, 0.1, 1, 10, 100],
              'svc__kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'svc__degree': [0.001, 0.01, 0.1, 1, 10, 100]
              }

# Perform grid search cross-validation to find the best hyperparameters
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and their corresponding accuracy
print("Best hyperparameters: ", grid_search.best_params_)
print("Best accuracy: ", grid_search.best_score_)

# Use the best hyperparameters to train the SVM model on the entire dataset
best_params = grid_search.best_params_
svm_model = SVC(C=best_params['svc__C'], kernel=best_params['svc__kernel'], degree=best_params['svc__degree'])
svm_model.fit(X_train, y_train)

# Evaluate the SVM model on the testing set and print the accuracy
y_pred = svm_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ", accuracy)
#predicting the validation set
y_pred_val = svm_model.predict(X_val)
accuracy_val = accuracy_score(y_val, y_pred_val)
print("Accuracy on validation data: ", accuracy_val)


Best hyperparameters:  {'svc__C': 10, 'svc__degree': 2, 'svc__kernel': 'rbf'}
Best accuracy:  0.9999849056603773
Accuracy:  1.0
Accuracy on validation data:  0.9995475113122172


In [20]:
from skimage import io
import cv2
from PIL import Image
import joblib
from skimage.color import rgb2gray
from skimage.feature import hog
import os

import numpy as np

# define border color
lower = (0, 80, 110)
upper = (0, 120, 150)
image_dir = '/Users/metoditarnev/Desktop/letters/resized_img_sorted'

# loop through images in directory
for filename in os.listdir(image_dir):
    if filename.endswith('.png'):
        image_path = f"{image_dir}/{filename}"
        img = io.imread(image_path)

        # threshold on border color
        mask = cv2.inRange(img, lower, upper)

        # dilate threshold
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 100))
        mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel)

        # recolor border to white
        img[mask==255] = (255,255,255)

        # convert img to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # otsu threshold
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_OTSU )[1]

        # apply morphology open
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,100))
        morph = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
        morph = 255 - morph

        # find contours and bounding boxes
        bboxes = []
        bboxes_img = img.copy()
        contours = cv2.findContours(morph, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours[0] if len(contours) == 2 else contours[1]
        for cntr in contours:
            x,y,w,h = cv2.boundingRect(cntr)
            cv2.rectangle(bboxes_img, (x, y), (x+w, y+h), (0, 0, 255), 1)
            bboxes.append((x,y,w,h))

        # get largest width of bboxes
        maxwidth = max(bboxes)[2]

        # sort bboxes on x coordinate
        def takeFirst(elem):
            return elem[0]

        bboxes.sort(key=takeFirst)

        # stack cropped boxes with 10 pixels padding all around
        result = np.full((1,maxwidth+10,3), (255,255,255), dtype=np.uint8)
        crops = []
        for bbox in bboxes:
            (x,y,w,h) = bbox
            crop = img[y:y+h, x:x+w]
            resized_crop = cv2.resize(crop, (100, 100))
            cv2.imwrite(f'/Users/metoditarnev/Desktop/letters/ImagesCVtest/{filename}', resized_crop)
